# Laboratorio 6
#### 20880 Sebastian Aristondo
#### 20293 Daniel Gonzalez

## Ejercicio 1

Cree un programa en Python (o en el lenguaje de su preferencia) para modelar la distribución de temperatura dentro
de una varilla sólida. Utilice modelado secuencial multiescala, donde combina un modelo macroscópico y un
modelo microscópico. Suponga que la varilla consta de dos materiales diferentes con diferentes conductividades
térmicas.

In [ ]:
import random

class atomo(object):
    
    def __init__(self, masa, left_neighbour, right_neighbour):
        self.move= ["left", "right"]
        self.mass = masa
        self.left_neighbour = self.left_neighbour
        self.right_neighbour = self.right_neighbour

    def random_move(self):
        self.position = random.choice(self.move)
    
    def exist_colision(self):
        if self.left_neighbour.position == "right" and self.position == "left":
            return (self.left_neighbour.mass + self.mass)*0.5
            
        if self.right_neighbour.position == "left" and self.position == "right":
            return (self.right_neighbour.mass + self.mass)*0.5
        
        return 0


In [ ]:
masa_aluminio = 2700
masa_hierro = 7850